<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/S_VS_T_FCNN_WORD_2_VEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import os, re
import tensorflow_hub as hub
import tensorflow as tf
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
os.environ['KAGGLE_USERNAME'] = "nikhileswarkomati"
os.environ['KAGGLE_KEY'] = "001b3a30170775e55950edb6ff0c9b17"
!kaggle datasets download -d nikhileswarkomati/suicide-watch

100% 115M/115M [00:01<00:00, 131MB/s] 
100% 115M/115M [00:01<00:00, 106MB/s]


In [ ]:
!unzip '/content/suicide-watch.zip'

Archive:  /content/suicide-watch.zip
  inflating: SuicideAndDepression_Detection.csv  


In [ ]:
data = pd.read_csv('/content/SuicideAndDepression_Detection.csv', lineterminator = '\n')
data.sample(5)

,text,class
140170,im so depressed and anxious I just can't take ...,depression
333257,I got a Christmas card from my grandma.It talk...,depression
171732,"Don't you dare tell me to ""Just snap out of it...",depression
287731,You know... there’s a chance you’ve whacked of...,teenagers
306716,Bruh I remember the r/teenagers GC on Discord ...,teenagers


In [ ]:
print(data.shape)
data['class'].value_counts()

(348111, 2)


teenagers       116037
depression      116037
SuicideWatch    116037
Name: class, dtype: int64

In [ ]:
# X = data.iloc[:, 0].values
# y = data.iloc[:, 1].values

# le = LabelEncoder()

# y = le.fit_transform(y)

# train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
# train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
# print(train_X.shape, test_y.shape, val_X.shape)

#------------------------------------------------------------------------------------------
X = data.loc[(data['class'] != 'depression'), 'text'].values
y = data.loc[(data['class'] != 'depression'), 'class'].values

le = LabelEncoder()

y = le.fit_transform(y)

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
print(train_X.shape, test_y.shape, val_X.shape)

(23207,) (116037,) (92830,)


In [ ]:
embeddings = embed(data.loc[(data['class'] != 'depression'), 'text'][:10000].values)
train_y = le.fit_transform(data.loc[(data['class'] != 'depression'), 'class'][:10000].values)
test_embeddings = embed(data.loc[(data['class'] != 'depression'), 'text'][10000:15000].values)
test_y = le.transform(data.loc[(data['class'] != 'depression'), 'class'][10000:15000].values)


# ------------------------------------------------------------------------------------
# embeddings = embed([preprocess(ele) for ele in data.loc[(data['class'] != 'depression'), 'text'][:10000].values])
# train_y = le.fit_transform(data.loc[(data['class'] != 'depression'), 'class'][:10000].values)
# test_embeddings = embed([preprocess(ele) for ele in data.loc[(data['class'] != 'depression'), 'text'][10000:15000].values])
# test_y = le.transform(data.loc[(data['class'] != 'depression'), 'class'][10000:15000].values)

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-05-19 08:12:38--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.88.149
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.88.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G   110MB/s    in 18s     

2021-05-19 08:12:56 (89.7 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
import gensim
from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format("/content/GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

In [ ]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        #logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [ ]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
for i in range(train_X.shape[0]):
    train_X[i] = w2v_tokenize_text(train_X[i])

for i in range(test_X.shape[0]):
    test_X[i] = w2v_tokenize_text(test_X[i])

In [ ]:
X_train_word_average = word_averaging_list(wv,train_X)
X_test_word_average = word_averaging_list(wv,test_X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  


In [ ]:
import itertools


%matplotlib inline


from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.preprocessing import text, sequence
from keras import utils
from keras.layers import LSTM
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer

checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)


model = Sequential()
model.add(Dense(512, input_shape = (300, )))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# num_classes = 2
# y_train = utils.to_categorical(train_y, num_classes)
# y_test = utils.to_categorical(test_y, num_classes)

batch_size = 256
epochs = 50



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())
history = model.fit(X_train_word_average, train_y,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=1,
                    callbacks = [checkpoint, earlystopping],
                    validation_split=0.1)


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 512)               154112    
_________________________________________________________________
activation_28 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 513       
_________________________________________________________________
activation_29 (Activation)   (None, 1)                 0         
Total params: 154,625
Trainable params: 154,625
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
82/82 [==============================] - 1s 12ms/step - loss: 0.5773 - accuracy: 0.7243 - val_los

In [ ]:
from keras.models import load_model
model = load_model('/content/model.h5')
y_pred = model.predict(X_test_word_average)
y_pred = np.rint(y_pred)
y_test = test_y
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.9077190896007308
              precision    recall  f1-score   support

         0.0       0.93      0.89      0.91     60141
         1.0       0.89      0.92      0.91     55896

    accuracy                           0.91    116037
   macro avg       0.91      0.91      0.91    116037
weighted avg       0.91      0.91      0.91    116037

